In [1]:
import osmium
import sys
from neo4j import GraphDatabase, basic_auth, unit_of_work
from copy import deepcopy

In [2]:
url = 'http://download.geofabrik.de/europe/monaco-latest.osm.pbf'

In [3]:
class GraphNode:
    def __init__(self, node_id, lat, long):
        self.node_id = node_id
        self.lat = lat
        self.long = long

    def __repr__(self):
        return f'{{node_id: {self.node_id}, lat: {self.lat}, long: {self.long}}}'

In [4]:
class GraphEdge:
    def __init__(self, start_node_id, end_node_id, distance, rating):
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.distance = distance
        self.rating = rating

In [5]:
class Neo4jHandler():
    def __init__(self):
        self.driver = GraphDatabase.driver('bolt://54.167.190.221:7687',
                                           auth=basic_auth(
                                               'neo4j',
                                               'release-shows-breads'))
        self.nodes = []
        self.edges = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def create_objects(self):
        with self.driver.session() as session:
            batch = []
            for i, node in enumerate(self.nodes):
                if (i % 4000 == 0 and len(batch) > 0) or i == len(self.nodes):
                    with session.begin_transaction() as tx:
                        tx.run(
                            f'''FOREACH (node IN [{','.join(map(lambda x: str(x), batch))}] |
                               CREATE (:Node{{node_id:node.node_id,lat:node.lat,long:node.long}}))'''
                        )
                        tx.commit()
                        print(f'Close transaction {i//5000}')
                        batch = []

                batch.append(node)
            for i, edge in enumerate(self.edges):
                print(i, edge.start_node_id, '->', edge.end_node_id)
                session.run(f'''MATCH (a: Node), (b: Node)
                    WHERE a.node_id = {edge.start_node_id} AND b.node_id = {edge.end_node_id}
                    CREATE (a)-[r:Route {{distance:$distance, rating:$rating}}]->(b)
                    ''',
                            distance=edge.distance,
                            rating=edge.rating)

    def close(self):
        self.driver.close()

In [6]:
class FileHandler(osmium.SimpleHandler):
    def __init__(self, neoHandler):
        super(FileHandler, self).__init__()
        self.neoHandler = neoHandler

    def node(self, n):
        id = deepcopy(n.id)
        lat, lon = deepcopy(str(n.location)).split('/')
        node = GraphNode(n.id, lat, lon)
        self.neoHandler.add_node(node)

    def way(self, w):
        distance = osmium.geom.haversine_distance(w.nodes)
        if w.nodes[0].ref == w.nodes[-1].ref:
            return
        edge = GraphEdge(w.nodes[0].ref, w.nodes[-1].ref, distance, 4)
        self.neoHandler.add_edge(edge)

In [17]:
driver = GraphDatabase.driver("bolt://54.167.190.221:7687",
                              auth=basic_auth("neo4j", "release-shows-breads"))

In [18]:
%%time

with driver.session() as session:
    session.run('MATCH (n) DETACH DELETE n')
neoHandler = Neo4jHandler()
h = FileHandler(neoHandler)

h.apply_file(url, locations=True)

neoHandler.create_objects()
neoHandler.close()

Close transaction 0
Close transaction 1
Close transaction 2
Close transaction 3
Close transaction 4
Close transaction 4
0 21912089 -> 21912099
1 21918500 -> 1868736741
2 25177418 -> 25177397
3 25192130 -> 1074585009
4 25201041 -> 25201047
5 25182821 -> 7932125562
6 25182815 -> 25182821
7 4938436902 -> 1079045416
8 25193709 -> 25194196
9 25181935 -> 2737814240
10 96050956 -> 25182815
11 25182446 -> 2153445075
12 25181793 -> 25181806
13 25191634 -> 25191338
14 25216581 -> 25191243
15 25191904 -> 1573123086
16 25191508 -> 25191502
17 1737389117 -> 477618046
18 25192216 -> 1074585009
19 25194260 -> 25194196
20 4058432473 -> 4058432482
21 1690130866 -> 1738360319
22 6444966511 -> 25193390
23 1918966551 -> 4940692951
24 1074584672 -> 4453073043
25 25177397 -> 25177819
26 25204258 -> 25177418
27 937988290 -> 1866510571
28 1868723951 -> 21918450
29 1720684257 -> 21912962
30 25197949 -> 25197962
31 25198656 -> 25238669
32 25238772 -> 25197962
33 25198559 -> 25238728
34 25238712 -> 25197985
35 2

301 1684697664 -> 1684697668
302 1684697627 -> 1720684198
303 1684697707 -> 1684697724
304 1684697693 -> 1684697699
305 1684697734 -> 1737147102
306 25240095 -> 1684697708
307 1684697654 -> 1684697669
308 1684697724 -> 1684697722
309 4417553808 -> 1685062069
310 1685062091 -> 1685062087
311 1685062094 -> 1685062091
312 5601785139 -> 1685132425
313 1685061918 -> 1685061915
314 1684697708 -> 1684697707
315 1685062074 -> 1685062058
316 1685132425 -> 7927968397
317 1736938093 -> 1736938084
318 1685062087 -> 1685062082
319 1685062069 -> 1685062074
320 1685062058 -> 1079750840
321 273246214 -> 1685061940
322 1685062007 -> 1685062030
323 1684697708 -> 21914666
324 1685062035 -> 1685062082
325 273246000 -> 21918825
326 1079751330 -> 1685062028
327 1699777673 -> 5603791857
328 1079751085 -> 1079751172
329 1685108373 -> 1685108369
330 1685108348 -> 1685108358
331 1685108397 -> 1685108389
332 1685108357 -> 1685108365
333 1685108329 -> 1685108357
334 1685108236 -> 1685108188
335 1685108264 -> 1685

592 25194212 -> 1759785785
593 25216769 -> 252362324
594 25193892 -> 25193802
595 25194143 -> 2270588149
596 3883559329 -> 25197470
597 252458682 -> 3416090602
598 1759785835 -> 1761115482
599 252458682 -> 21917628
600 4970587876 -> 4970587862
601 252474584 -> 1759785856
602 1789813053 -> 1759785866
603 3805333988 -> 21928938
604 280487754 -> 25181935
605 1800775440 -> 25181935
606 25193617 -> 1870384826
607 273246269 -> 273246214
608 25240089 -> 1868723931
609 21918450 -> 1868723951
610 21914339 -> 1738360276
611 1770577842 -> 1352289907
612 21915639 -> 25240089
613 21917586 -> 3883559329
614 25201044 -> 1696714161
615 1868723949 -> 21918402
616 25238712 -> 25238728
617 25195773 -> 25197949
618 25238728 -> 25238772
619 25177449 -> 1204303579
620 25177185 -> 1778433989
621 25177418 -> 2104729494
622 1074585009 -> 6444966511
623 1738360319 -> 25193377
624 1347559085 -> 937988407
625 1738360276 -> 1690130866
626 252356770 -> 1074584855
627 1696644216 -> 25210895
628 25210837 -> 186876779

885 1878520653 -> 3266145216
886 1878520547 -> 1878520635
887 1878520646 -> 1878520658
888 1079750543 -> 1878520646
889 1878520940 -> 1878520914
890 1868723942 -> 1878539785
891 1347117298 -> 1347117299
892 1878844638 -> 1347117287
893 1878844639 -> 1347117271
894 1878844714 -> 1878844698
895 1878844720 -> 1878844681
896 1878844696 -> 1878844667
897 1879687757 -> 1878539803
898 1685108392 -> 1879687763
899 1685108394 -> 1879687797
900 25197970 -> 25197985
901 25243371 -> 1880126682
902 1880126683 -> 1880126681
903 7841390730 -> 1880130279
904 1880130279 -> 1880130277
905 21919273 -> 1880130278
906 1868752099 -> 1882366970
907 1882366970 -> 1868752089
908 1352205619 -> 251721708
909 1784106882 -> 25201041
910 3419409376 -> 1784106966
911 1784106966 -> 252362102
912 1702432008 -> 1702432018
913 25177743 -> 1930502733
914 25177743 -> 1204303579
915 1930521959 -> 1930521958
916 1930521958 -> 7788219292
917 25191502 -> 25191494
918 1690189849 -> 1930545590
919 1930545590 -> 4424273190
920 2

1167 2229413963 -> 4453063928
1168 3646650957 -> 252358384
1169 252356760 -> 3646650948
1170 3646650958 -> 3646650943
1171 3646667842 -> 3646667844
1172 3647254919 -> 3647254920
1173 252539557 -> 3647254919
1174 252362088 -> 25202530
1175 1784106819 -> 3789557768
1176 1696644337 -> 3419584652
1177 3949642984 -> 1696644284
1178 3949642994 -> 1696644276
1179 3949643004 -> 3949643030
1180 3949643007 -> 1681932258
1181 25193570 -> 25193617
1182 4011267552 -> 25193570
1183 4011267563 -> 4011267556
1184 2109529537 -> 1866510556
1185 4011307652 -> 4011267563
1186 1079750723 -> 1079751625
1187 4011359448 -> 2330295695
1188 1079751625 -> 2330295695
1189 4011359448 -> 443322800
1190 443322858 -> 443322800
1191 4011405464 -> 4011405441
1192 4011405463 -> 4011405442
1193 4011405442 -> 4011405441
1194 25178405 -> 25178386
1195 25178371 -> 25178386
1196 25178405 -> 25178371
1197 4029620782 -> 4029620783
1198 4029620781 -> 4029620782
1199 4029620783 -> 4029620784
1200 4029620777 -> 5181191623
1201 40

1444 1875160748 -> 1875160729
1445 4939779360 -> 1875160748
1446 1868008127 -> 25177834
1447 4939779371 -> 1074584775
1448 7781960093 -> 7781960097
1449 4939779423 -> 4939779404
1450 4939779409 -> 4939779406
1451 4939779410 -> 4939779415
1452 4939779415 -> 5918147599
1453 1869347215 -> 1868736596
1454 1869347242 -> 4940249417
1455 4940249424 -> 4937466461
1456 4937466461 -> 2109529551
1457 2109529551 -> 2109571494
1458 1079750403 -> 21919238
1459 273246003 -> 21918825
1460 4940452496 -> 273244852
1461 4439411462 -> 4940452506
1462 4940452514 -> 4940452518
1463 4940452734 -> 4940452728
1464 4940452728 -> 4940452729
1465 21919238 -> 4940452734
1466 4940452735 -> 4940452736
1467 4940452738 -> 29457984
1468 4940452746 -> 1079750314
1469 1870384824 -> 1870384826
1470 1712696757 -> 1800775440
1471 1870354490 -> 6458602911
1472 1712696755 -> 1784106917
1473 1074584632 -> 2104842206
1474 1784106917 -> 8047801201
1475 4937556546 -> 25182101
1476 1204288473 -> 25191904
1477 1573123086 -> 2519184

1719 6445480796 -> 2605737762
1720 6445480810 -> 6445480812
1721 6445480813 -> 6445480815
1722 5504668074 -> 6445480816
1723 6445480816 -> 6445480817
1724 6445480817 -> 6445480818
1725 25216571 -> 6445714057
1726 6445714057 -> 6445714059
1727 1639786235 -> 1639786209
1728 1868015873 -> 6453128380
1729 6453128373 -> 6453128380
1730 6453128380 -> 6453128383
1731 6453128380 -> 6454011643
1732 6453128380 -> 6453128487
1733 6453128373 -> 1639786235
1734 6456373620 -> 918052367
1735 6456373625 -> 6456373626
1736 6456373627 -> 6456373628
1737 1702432008 -> 6456373640
1738 6456377964 -> 7788176349
1739 4937466478 -> 6456377965
1740 6456377965 -> 6456377964
1741 1074584836 -> 6456543857
1742 6456543857 -> 6456543858
1743 4100895547 -> 6445217043
1744 6456631174 -> 6445217046
1745 1878520843 -> 1878520849
1746 1784106906 -> 6458472571
1747 6458472580 -> 1784106905
1748 6458472581 -> 1784106907
1749 6458472584 -> 6458473289
1750 6458473290 -> 1784106902
1751 1784106903 -> 6458473293
1752 64585119

1994 6780002020 -> 6780001467
1995 6780001467 -> 6780004283
1996 6780008890 -> 6780005884
1997 6780004563 -> 6780006493
1998 6780005884 -> 6780002020
1999 6780008159 -> 6780002020
2000 6780004283 -> 6780004563
2001 6780001467 -> 6780005353
2002 6845638316 -> 6845638317
2003 6845638317 -> 6845638318
2004 6845638318 -> 6848669738
2005 252362111 -> 252362112
2006 6846911425 -> 6846911427
2007 6846911429 -> 6846911431
2008 1784106954 -> 252539514
2009 1784106954 -> 3428355440
2010 6846911431 -> 3419409376
2011 6846911429 -> 6506285019
2012 6459866642 -> 6459866645
2013 6459866643 -> 6459866639
2014 6847756091 -> 6847756092
2015 6847756092 -> 6846911427
2016 6847727077 -> 5181191623
2017 6848669737 -> 6848669736
2018 6848669738 -> 6848669737
2019 1079750499 -> 6848754637
2020 6848754637 -> 6848754641
2021 7788176324 -> 7788176318
2022 6848754645 -> 1079750499
2023 6848754645 -> 6848754646
2024 1702432036 -> 6848754646
2025 6850889195 -> 6850889196
2026 6850889196 -> 6850889197
2027 68509334

2270 7788176322 -> 7788176324
2271 7788176286 -> 7788176304
2272 7788176295 -> 7788176329
2273 7788176330 -> 917978743
2274 917978757 -> 7788176332
2275 7799413968 -> 7788176342
2276 7788176346 -> 7788176347
2277 7788176332 -> 7799413968
2278 7788176339 -> 7788176352
2279 7792524074 -> 7788214974
2280 1079750255 -> 7788214984
2281 2737814229 -> 7788176349
2282 2737814234 -> 7787103272
2283 7788214984 -> 7788219286
2284 7788219289 -> 3266162060
2285 7788219292 -> 7788219293
2286 7788219300 -> 7788219302
2287 7788219303 -> 7788219304
2288 7791022981 -> 7791022982
2289 7791022982 -> 4939779416
2290 5918361255 -> 5918147593
2291 7791036014 -> 7791036015
2292 7781960111 -> 7791036022
2293 7792524074 -> 7792524075
2294 7792524075 -> 7792524076
2295 7796549686 -> 3068739807
2296 7796549688 -> 7796549686
2297 7797546921 -> 7797546922
2298 7797546923 -> 7797546924
2299 1079751389 -> 7799231114
2300 7799519347 -> 7799519348
2301 7799519348 -> 7799519355
2302 7799519356 -> 7799519357
2303 7799519

In [9]:
nodes = []
with driver.session() as session:
    result = session.run('MATCH (n:Node) return n')
    for record in result:
        nodes.append([record[0].get('lat'), record[0].get('long')])

In [11]:
import numpy as np


def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

In [15]:
arr = np.array([[10, 2], [12, -2], [3, 8]])
err = (arr - np.array([12, 4]))**2
delta = np.sum(err, axis=1)
pos = np.argmin(delta)
print(err, delta, pos)

[[ 4  4]
 [ 0 36]
 [81 16]] [ 8 36 97] 0
